#### Importownie bibliotek

In [1]:
import pandas as pd
import numpy as np
import pickle

import xgboost as xgb

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

#### Wczytanie dev_df z pliku

In [2]:
dev_df=pd.read_csv("dev_df.csv")

x_dev=dev_df.drop(['pledged_in_usd','percentage_of_money_collected','backers'], axis=1)
y_dev=dev_df['pledged_in_usd']

#### Reprezentacja x i y 

In [3]:
x_dev.head()

,main_cat_cat,country,duration,currency,goal_in_usd,state
0,80,21,30,13,500.00,1
1,54,7,30,5,3315.48,0
2,0,21,31,13,18000.00,1
3,154,21,45,13,60000.00,0
4,159,21,45,13,75000.00,0


In [4]:
y_dev.head()

0      575.00
1       33.15
2    20891.00
3        0.00
4    11734.01
Name: pledged_in_usd, dtype: float64

#### Podział próbek na treningowe i testowe

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_dev, y_dev, test_size=0.2)

#### Podanie parametrów do xgb.XGBRegressor

In [6]:
params= {'objective': 'reg:squarederror'}

#### Wyliczenie modelu xgb.XGBRegressor

In [7]:
def test(params):
    model = xgb.XGBRegressor(**params)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2score = r2_score(y_test, y_pred)
    score = model.score(x_train,y_train)
    
    return mse, mae, r2score, score, model

#### Wyniki

In [8]:
mse, mae, r2, score, model = test(params)
print('\nType: XGBRegressor', '\nParams: ', params, '\nMse: ',mse,'\nMae: ', mae, '\nR2: ',r2,'\nScore:',score)

/home/nina/miniconda3/envs/m1/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



Type: XGBRegressor 
Params:  {'objective': 'reg:squarederror'} 
Mse:  2349920357.302324 
Mae:  8011.710833275488 
R2:  0.4341620250340311 
Score: 0.2809087915147672


#### Zapisanie modelu wyliczonego z dev_df

In [9]:
pkl_filename = "model.pickle"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

#### W celu pobrania modelu z pliku wystarczy tyle:

In [10]:
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

#### Sprawdzenie czy się poprawnie otworzył

In [11]:
pickle_model
y_pred = pickle_model.predict(x_test)
    
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2score = r2_score(y_test, y_pred)
score = pickle_model.score(x_train,y_train) 
mse, mae, r2score, score

(2349920357.302324, 8011.710833275488, 0.4341620250340311, 0.2809087915147672)